## Overview
This notebook implements the model selection phase of the example. To explain the patterns of daily customer purchases
at the store we run PCA on the trimmed daily sales representation. There are many choices to explain variation and summarize
data. PCA is used here for illustration. Other choices such as support vector data description or matrix factorization based approaches
could be favored by some analysts or teams. The point here is that it is possible to capture annotations about rationale used
to make model selection decisions. This is done after developing the model and logging the observations to KMDS. The basic recipe in the note book is:
1. Read the data representation post the modelling choice decision to trim the representation to only use store inventory that accounts for daily variation - see the modelling choice notebook for details
2. Run PCA on the representation
3. Analyze the results
4. Log the observations with KMDS

## Read the Data

In [1]:
import pandas as pd
fp = "../../kmds/examples/retail_q1_post_data_rep_prep.parquet"
df = pd.read_parquet(fp)

## Run PCA

In [2]:
from sklearn.decomposition import PCA

In [3]:
retail_model = PCA(n_components=30)
retail_model.fit_transform(df)

array([[-344.71612516,  -53.99743665,  -67.59587817, ..., -287.95692438,
         277.46668133,  136.53220333],
       [-197.1269941 ,   44.14861636, -120.21414823, ...,  224.70953596,
          98.08817544,  172.87088799],
       [-336.60229492,   20.45967915, -100.43399668, ...,  -42.17944044,
         -12.39667032,    4.63986837],
       ...,
       [ 165.28816272,  152.77402686, -161.67343892, ...,   44.75652174,
          15.23219448,    2.88565728],
       [-235.02850992,  102.91634809,  -78.24785667, ...,  -11.16375464,
          42.47323285,  -53.61650486],
       [ -94.96493568,  109.86188469,  -67.86396433, ...,   37.34952601,
          -3.07720408,   24.3452214 ]])

In [4]:
 retail_model.explained_variance_ratio_.cumsum()

array([0.12280201, 0.21023078, 0.29474311, 0.35655678, 0.40462036,
       0.44959697, 0.4931996 , 0.52644409, 0.5544069 , 0.58047431,
       0.60502988, 0.62725672, 0.64848516, 0.66944554, 0.68880626,
       0.70562975, 0.72216695, 0.73750766, 0.75260065, 0.76592016,
       0.77847674, 0.78978961, 0.80070352, 0.81082625, 0.82073043,
       0.83011091, 0.83874391, 0.84689601, 0.8546312 , 0.86209863])

## Log the Observations to KMDS

In [5]:
from tagging.tag_types import *
from owlready2 import *
from utils.load_utils import *
KNOWLEDGE_BASE = "example_ml_kb_exp_workflow.xml"


In [6]:
onto2 = load_kb(KNOWLEDGE_BASE)
with onto2:
    insts = Workflow.instances()
the_workflow_instance = insts[0]

In [7]:
the_workflow_instance

example_ml_kb_exp_workflow.xml.retail_customer_modelling

In [8]:
ms_obs_list = []
observation_count = 1

ms1 = ModelSelectionObservation(namespace=onto2)
ms1.finding = "For this iteration, PCA was the only modelling approach to explain the variance in daily\
product sales. This is sufficient to illustrate how a model selection workflow is logged with KMDS."
ms1.finding_sequence = observation_count
ms1.model_selection_observation_type = ModelSelectionTags.MODEL_SELECTION_STATEMENT.value
ms_obs_list.append(ms1)
the_workflow_instance.has_model_selection_observations = ms_obs_list


onto2.save(file=get_kb_file_path(KNOWLEDGE_BASE), format="rdfxml")